In [1]:
import scipy
import scipy.stats
import pandas as pd
import numpy as np
from tqdm import tqdm
import time

In [2]:
df = pd.read_pickle('firehoseTAZ')

In [3]:
len(df)

111301

In [4]:
df.head()

,cate,day_of_week,hour_of_day,minute_of_day,TAZ
0,Outdoors & Recreation,3,2,127,40
1,Professional & Other Places,3,7,478,40
2,Food,3,9,546,40
3,Food,3,9,553,40
4,Food,3,9,553,40


In [38]:
sequenceLength=10
timeLimit=60
tazCount=318
timeCount=1440

In [39]:
arrival_t=np.zeros((tazCount,timeCount))

In [40]:
for i in tqdm(range(len(df))):
    arrival_t[df.loc[i,'TAZ']-1][df.loc[i,'minute_of_day']]+=1

100%|███████████████████████████████| 111301/111301 [00:01<00:00, 60141.65it/s]


In [41]:
arrival_t_pad=np.asarray([np.append(x,x[:sequenceLength+timeLimit]) for x in arrival_t])

In [42]:
arrival_t_pad.shape

(318, 1510)

In [43]:
sequence_t=[]

In [44]:
for i in range(tazCount):
    for t in range(timeCount+timeLimit):
        sequence_t.append(arrival_t_pad[i][t:t+sequenceLength])    

In [45]:
friction=np.zeros((tazCount,tazCount))

In [60]:
np.var([1,1,1])

0.0

In [62]:
for i in tqdm(range(tazCount)):
    for t in range(0,timeCount,sequenceLength):
        #start_t_time = time.time()
        if np.var(sequence_t[i*timeCount+t])!=0:
            x=sequence_t[i*timeCount+t]
            sequence_t_dataframe=np.array([sequence_t[j*timeCount+s] for j in range(tazCount) for s in range(t,t+timeLimit,sequenceLength)])
            for k in range(timeLimit*tazCount//sequenceLength):
                if np.var(sequence_t_dataframe[k])!=0:
                    friction[i][k//(timeLimit//sequenceLength)]=friction[i][k//(timeLimit//sequenceLength)]+\
                    np.nan_to_num(scipy.stats.pearsonr(x, sequence_t_dataframe[k])[0])
        #end_t_time = time.time()
    print (i)



  0%|                                                  | 0/318 [00:00<?, ?it/s]

0



  0%|▏                                         | 1/318 [00:06<32:44,  6.20s/it]
Exception in thread Thread-9:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda2\envs\tf\lib\threading.py", line 914, in _bootstrap_inner
    self.run()
  File "C:\ProgramData\Anaconda2\envs\tf\lib\site-packages\tqdm\_tqdm.py", line 148, in run
    for instance in self.tqdm_cls._instances:
  File "C:\ProgramData\Anaconda2\envs\tf\lib\_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



1


  1%|▎                                         | 2/318 [00:16<42:27,  8.06s/it]

...

3


  1%|▌                                         | 4/318 [00:25<33:40,  6.43s/it]

4


  2%|▋                                         | 5/318 [00:36<38:03,  7.30s/it]


KeyboardInterrupt: 